In [ ]:
pip install pennylane

# Helpers

In [ ]:
import numpy as np

class GaussianProcessRegressor:
    """
    Gaussian Process Regressor for d-dimensional inputs.
    Returns both predictive mean and std if requested.
    """
    def __init__(self, kernel, alpha=1e-5):
        self.kernel = kernel
        self.alpha = alpha
        self.X_train = None
        self.y_train = None
        self.K_inv = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.X_train = X
        self.y_train = y
        K = self.kernel(X, X)
        K += self.alpha * np.eye(len(X))
        try:
          self.K_inv = np.linalg.inv(K)
        except:
          print("error")
          self.K_inv = np.linalg.pinv(K)

    def predict(self, X_test, return_std=False):

        X_test = np.array(X_test)
        K_star = self.kernel(X_test, self.X_train)
        y_mean = K_star @ (self.K_inv @ self.y_train)

        if return_std:
            K_star_star = self.kernel(X_test, X_test)
            cov = K_star_star - K_star @ self.K_inv @ K_star.T
            var = np.diag(cov)
            var = np.maximum(var, 0.0)
            y_std = np.sqrt(var)
            return y_mean, y_std
        else:
            return y_mean



from itertools import product

def build_grid(bounds, n_grid):
    """
    Build a uniform grid of points within the given 'bounds'.

    Parameters
    ----------
    bounds : list of (low, high) for each dimension (length d)
    n_grid : int
        Number of grid points per dimension.

    Returns
    -------
    X_grid : np.ndarray of shape (n_grid^d, d)
        All points in the grid.
    """
    # For each dimension, create an array of n_grid points from low to high
    axes = [np.linspace(low, high, n_grid) for (low, high) in bounds]
    # Cartesian product of all axes
    # e.g. for d=2, we get all pairs (x,y); for d=3, all (x,y,z), etc.
    mesh = list(product(*axes))  # a list of d-tuples
    return np.array(mesh)


import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from scipy.stats import norm
import random


def gp_ucb_nd(
    f,                 # black-box reward function, shape: (n, d) -> (n,)
    gp,                # a GaussianProcessRegressor instance (fit, predict)
    bounds,            # list of (low, high) for each dimension
    n_iter=10,         # number of UCB iterations
    init_points=3,     # number of initial random samples
    n_grid=50,         # number of grid points per dimension to search for UCB max
    beta_func=None,    # a callable or None => we define a default
    random_state=None,
    error = 1,
    verbose=False
):
    """
    Runs GP-UCB for a multi-armed bandit / Bayesian optimization problem.

    Parameters
    ----------
    f : callable
        The unknown reward function (black-box).
        Input shape (n, d) -> (n,) for n points in dimension d.
    gp : GaussianProcessRegressor
        Surrogate GP model implementing:
          - gp.fit(X, y)
          - gp.predict(X, return_std=True) -> (mean, std)
    bounds : list of (low, high)
        Domain bounding box in d dimensions.
    n_iter : int
        Number of GP-UCB rounds.
    init_points : int
        How many random points to sample initially.
    n_grid : int
        Grid resolution for selecting x_t = argmax_x [mu_{t-1}(x) + sqrt(beta_t)*sigma_{t-1}(x)].
    beta_func : callable or None
        If None, we define a simple default that grows with t.  Otherwise,
        something like:  beta_func(t) -> float
    random_state : int or None
        For reproducibility.
    verbose : bool
        Print iteration details if True.

    Returns
    -------
    regrets : ndarray of shape (n_iter + init_points,)
        Cumulative regret at each iteration (the sum over t=1..T of [f(x*) - f(x_t)]).
    X_samples : ndarray of shape (n_iter + init_points, d)
    Y_samples : ndarray of shape (n_iter + init_points,)

    Notes
    -----
    - We assume we can approximate the global max f(x*) by a dense grid search
      once, for regret calculation.  In a real bandit scenario, you might not
      know x*, but for synthetic tests or known benchmark functions, we do.
    - The domain is searched by a grid of size (n_grid^d).  This is only feasible
      for small d or moderate n_grid.
    """

    if random_state is not None:
        np.random.seed(random_state)

    d = len(bounds)

    # 1) Create a dense grid to:
    #    - approximate x* (the global maximizer)
    #    - search for the UCB argmax each iteration
    X_grid = build_grid(bounds, n_grid=n_grid)  # shape (n_grid^d, d)

    # 2) (Optional) approximate the global maximum for regret calculation
    #    We take the best among the same grid points to get x_star
    Y_grid = f(X_grid)
    idx_best = np.argmax(Y_grid)
    x_star = X_grid[idx_best]
    f_star = Y_grid[idx_best]  # approximate global max value
    if f_star == 0:
      print("f_star is zero???")

    # 3) Helper to define a default beta_t if user didn't supply one
    if beta_func is None:
        # E.g. a common choice for finite domain or small bounding
        # can be something like: 2 ln(t^2 * pi^2 / (6 delta))
        # but we just do a simpler scaling of log t for demonstration.
        def beta_func(t):
            return 2.0 * np.log(1000 * t**2 * 10 / 6 / 0.1 )   # delta = 0.1
    # else user supplies something like beta_func(t) => some formula

    # 4) Initialize data by sampling 'init_points' random points
    def sample_random(n):
        return np.array([
            X_grid[random.sample(range(1, 10),1)[0]]
            for _ in range(n)
        ])

    X_samples = sample_random(init_points)  # shape (init_points, d)
    Y_samples = f(X_samples)               # shape (init_points,)

    # 5) We'll keep track of cumulative regrets
    regrets = np.zeros(n_iter + init_points)
    # First 'init_points' regrets are computed from those random picks
    # Evaluate the regret at each step
    cum_regret = 0.0
    for i in range(init_points):
        cum_regret += (f_star - Y_samples[i])
        regrets[i] = cum_regret

    # 6) Main loop of GP-UCB
    log_sum_for_beta = 0
    for step in range(n_iter):
        #print(step)
        t = init_points + step + 1  # total iteration index (1-based)

        # Fit GP on current data
        gp.fit(X_samples, Y_samples)

        # Compute mean & std on the entire grid
        mu_grid, std_grid = gp.predict(X_grid, return_std=True)

        # Define current beta_t
        beta_t = 0.1 * (2*np.log(20)+log_sum_for_beta) ** 0.5 + 3

        #beta_func(t)

        # UCB = mu + sqrt(beta_t) * std
        ucb_values = mu_grid + (beta_t + error * np.sqrt(t) ) * std_grid

        # if step % 10 == 0:
        #   print(step)
        #   print(mu_grid[:10], std_grid[:10])
        #   print(ucb_values[:10])
        #   #print(np.max(ucb_values))
        #   print("==========")

        # Argmax on the grid
        idx_next = np.argmax(ucb_values)
        x_next = X_grid[idx_next].reshape(1, -1)

        log_sum_for_beta += np.log(1 + std_grid[idx_next])

        #print(beta_t + error * np.sqrt(t) )
        #print(std_grid[idx_next])

        # Evaluate the unknown function f (bandit feedback)
        np.random.seed(None)
        noise = np.random.normal(0, 0.1)
        y_true = f(x_next)
        y_next = y_true + noise
        #print(noise)

        # Update data
        X_samples = np.vstack([X_samples, x_next])
        Y_samples = np.concatenate([Y_samples, y_next])

        # Update cumulative regret
        cum_regret += (f_star - y_true[0])
        regrets[init_points + step] = cum_regret

        if verbose:
            print(f"Iteration {step+1}/{n_iter}, t={t}, beta={beta_t:.3f}, "
                  f"x_next={x_next[0]}, f(x)={y_next[0]:.4f}, UCB={ucb_values[idx_next]:.4f}, "
                  f"CumReg={cum_regret:.4f}")

    return regrets, X_samples, Y_samples





import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

def run_multiple_experiments_gp_ucb(
    f,
    gp,
    bounds,
    n_runs=5,
    n_iter=10,
    init_points=3,
    n_grid=50,
    beta_func=None,
    random_state=None,
    error = 1,
    verbose=False
):
    """
    Runs the GP-UCB experiment 'n_runs' times, each time creating a new GP instance.
    Returns the average cumulative regret across runs.

    Parameters
    ----------
    f : callable
        The black-box reward function.
    gp_class_factory : callable
        A function that, when called, returns a *new* untrained GaussianProcessRegressor
        (or similar). For example:
          lambda: GaussianProcessRegressor(kernel=RBF(...), alpha=..., optimizer=None)
        We need a fresh GP for each run, so that each run is independent.
    bounds : list of (low, high)
        Domain bounding box.
    n_runs : int
        Number of independent runs to average over.
    n_iter : int
        Number of GP-UCB rounds (not counting the init_points).
    init_points : int
        Number of random initial points in each run.
    n_grid : int
        Grid resolution for argmax search in each run.
    beta_func : callable or None
        If None, use a default log-based. Otherwise a function beta_func(t) -> float.
    random_state : int or None
        For reproducibility. If set, seeds the first run's RNG, then subsequent runs
        will shift the seed.
    verbose : bool
        Whether to print details for each run.

    Returns
    -------
    avg_regret : ndarray of shape (n_iter + init_points,)
        The pointwise average of the cumulative regret across runs.
    regrets_all : ndarray of shape (n_runs, n_iter + init_points)
        The individual run's cumulative-regret curves.
    """

    from copy import deepcopy

    # If we want reproducibility, set base seed
    base_seed = random_state if random_state is not None else None

    # We'll store the regret curve for each run here
    regrets_all = []

    for i in trange(n_runs, desc="GP-UCB Experiments"):
        # For each run, optionally shift the seed
        if base_seed is not None:
            # shift by i to get distinct seeds
            np.random.seed(base_seed + i)

        # Create a fresh GP instance
        gp_model = gp

        # Import or copy the gp_ucb_nd from your previous code snippet:
        regrets, X_samples, Y_samples = gp_ucb_nd(
            f=f,
            gp=gp_model,
            bounds=bounds,
            n_iter=n_iter,
            init_points=init_points,
            n_grid=n_grid,
            beta_func=beta_func,
            random_state=None,  # we've set the seed externally
            error = error,
            verbose=(verbose and i == 0)  # only verbose in the 1st run, e.g.
        )
        print(regrets)
        regrets_all.append(regrets)

    regrets_all = np.array(regrets_all)  # shape (n_runs, n_steps)
    avg_regret = regrets_all.mean(axis=0)
    std_regret = regrets_all.std(axis=0)
    return avg_regret, regrets_all, std_regret





## build kernels

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg as spla

##############################################################################
# 1) BINARY LABEL => label=1 if (above_l1 & below_l2), else label=0
##############################################################################

def binary_label_function(J1, J2):
    below_l1= (J2 < -1 - J1)
    above_l1= (J2 > -1 - J1)
    above_l2= (J2 >  J1 - 1)
    below_l2= (J2 <  J1 - 1)
    # ignoring l3=1

    if (above_l1 and below_l2):
        return 1
    else:
        return 0

##############################################################################
# 2) Cluster Hamiltonian => ground state
##############################################################################

def build_cluster_hamiltonian(n_qubits, J1, J2):
    def mod_idx(k):
        return k % n_qubits

    coeffs=[]
    ops=[]
    for j in range(n_qubits):
        ops.append(qml.PauliZ(j))
        coeffs.append(1.0)

        ops.append(qml.PauliX(j) @ qml.PauliX(mod_idx(j+1)))
        coeffs.append(-J1)

        ops.append(qml.PauliX(mod_idx(j-1)) @ qml.PauliZ(j) @ qml.PauliX(mod_idx(j+1)))
        coeffs.append(-J2)

    return qml.Hamiltonian(coeffs, ops)

def ground_state_exact(n_qubits, J1, J2):
    H= build_cluster_hamiltonian(n_qubits, J1, J2)
    sm= H.sparse_matrix()
    vals, vecs= spla.eigsh(sm, k=1, which='SA')
    return vecs[:,0]

##############################################################################
# 3) 2-layer circuit => measure qubit0 => [p0,p1]
##############################################################################

def multi_layer_circuit(params, n_qubits, data_vec, n_layers=2):
    qml.StatePrep(data_vec, wires=range(n_qubits))

    offset=0
    for _ in range(n_layers):
        # single-qubit
        for i in range(n_qubits):
            rx= params[offset+2*i]
            rz= params[offset+2*i+1]
            qml.RX(rx, wires=i)
            qml.RZ(rz, wires=i)
        offset+= 2*n_qubits

        # ring of cnot
        for i in range(n_qubits):
            qml.CNOT(wires=[i,(i+1)%n_qubits])

    return qml.probs(wires=[0])  # => [p0,p1]

def create_classifier_qnode(n_qubits, n_layers=2):
    dev= qml.device("default.qubit", wires=n_qubits)
    total_params= 2*n_qubits*n_layers

    @qml.qnode(dev)
    def classifier(params, data_vec):
        return multi_layer_circuit(params, n_qubits, data_vec, n_layers=n_layers)

    return classifier, total_params

##############################################################################
# 4) cost => 1 - average p_correct => label in {0,1}
##############################################################################

def empirical_risk(params, classifier_circuit, states, labels):
    N= len(states)
    sum_p= 0.0
    for i in range(N):
        lbl= labels[i]
        p2= classifier_circuit(params, states[i])
        sum_p+= p2[int(lbl)]
    return sum_p/N

def cost_fn(params, classifier_circuit, states, labels):
    return 1.0 - empirical_risk(params, classifier_circuit, states, labels)

def accuracy(params, classifier_circuit, states, labels):
    correct= 0
    for s,lbl in zip(states, labels):
        p2= classifier_circuit(params, s)
        pred_idx= int(np.argmax(p2))
        if pred_idx==lbl:
            correct+=1
    return correct/ len(states)

##############################################################################
# 5) data grid => STILL returns ground states in `states`,
#    used for training.  We'll keep this the same.
##############################################################################

def generate_data_grid(n_qubits, Nx, Ny, j1min, j1max, j2min, j2max, label_func):
    j1_vals= np.linspace(j1min, j1max, Nx)
    j2_vals= np.linspace(j2min, j2max, Ny)
    states=[]
    labels=[]
    j1_collect=[]
    j2_collect=[]
    for j1 in j1_vals:
        for j2 in j2_vals:
            st= ground_state_exact(n_qubits, j1, j2)
            lab= label_func(j1,j2)
            states.append(st)
            labels.append(lab)
            j1_collect.append(j1)
            j2_collect.append(j2)

    states= np.array(states, dtype=object)
    labels= np.array(labels, dtype=int)
    j1_collect= np.array(j1_collect)
    j2_collect= np.array(j2_collect)
    return states, labels, j1_collect, j2_collect

##############################################################################
# 6) QNode that returns a partial or full-wire density matrix, given ground state
##############################################################################

def create_density_qnode(n_qubits, n_layers=2, wires_to_measure=None):
    if wires_to_measure is None:
        wires_to_measure= list(range(n_qubits))

    dev= qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def density_circuit(params, ground_state):
        offset= 0
        qml.StatePrep(ground_state, wires=range(n_qubits))
        for _ in range(n_layers):
            for i in range(n_qubits):
                rx= params[offset+2*i]
                rz= params[offset+2*i+1]
                qml.RX(rx, wires=i)
                qml.RZ(rz, wires=i)
            offset+= 2*n_qubits
            for i in range(n_qubits):
                qml.CNOT(wires=[i,(i+1)%n_qubits])

        return qml.density_matrix(wires=wires_to_measure)

    return density_circuit

##############################################################################
# 7) KERNEL: minimal changes => now they accept Nx2 for (J1,J2)
#    so inside the kernel, we build ground states on-the-fly
##############################################################################

def build_kernel_full(n_qubits, n_layers, final_params):
    """
    kernel_full(Xa, Xb):
       - Xa, Xb have shape (N,2), (M,2). each row => [J1, J2].
       - build ground state => pass to QNode => get full-wire density => overlap
    """
    dens_qnode= create_density_qnode(n_qubits, n_layers=n_layers, wires_to_measure=None)

    def kernel_fn(Xa, Xb):
        N= Xa.shape[0]
        M= Xb.shape[0]
        K= np.zeros((N,M))

        for i in range(N):
            j1_i, j2_i= Xa[i,0], Xa[i,1]
            gs_i= ground_state_exact(n_qubits, j1_i, j2_i)
            rho_i= dens_qnode(final_params, gs_i)

            for j in range(M):
                j1_j, j2_j= Xb[j,0], Xb[j,1]
                gs_j= ground_state_exact(n_qubits, j1_j, j2_j)
                rho_j= dens_qnode(final_params, gs_j)
                K[i,j]= np.real(np.trace(rho_i@rho_j))

        return K

    return kernel_fn

def build_kernel_qubit(n_qubits, n_layers, final_params, qubit_index=[0]):
    """
    single-qubit partial => measure only qubit_index => 2x2
    """
    dens_qnode= create_density_qnode(n_qubits, n_layers=n_layers, wires_to_measure=qubit_index)

    def kernel_fn(Xa, Xb):
        N= Xa.shape[0]
        M= Xb.shape[0]
        K= np.zeros((N,M))

        for i in range(N):
            j1_i, j2_i= Xa[i,0], Xa[i,1]
            gs_i= ground_state_exact(n_qubits, j1_i, j2_i)
            rho_i= dens_qnode(final_params, gs_i)

            for j in range(M):
                j1_j, j2_j= Xb[j,0], Xb[j,1]
                gs_j= ground_state_exact(n_qubits, j1_j, j2_j)
                rho_j= dens_qnode(final_params, gs_j)
                K[i,j]= np.real(np.trace(rho_i@rho_j))

        return K

    return kernel_fn

##############################################################################
# 8) Minimal kernel ridge function => for demonstration
##############################################################################

def kernel_ridge_mse(X, y, kernel_fn, alpha=1e-6):
    """
    We do in-sample fit => y_pred => measure MSE
    X => Nx2, each row => (J1,J2)
    y => Nx
    kernel_fn => a function that does K(Xa, Xb)
    """
    N= len(X)
    K= kernel_fn(X, X)
    # K += alpha*np.eye(N)
    alpha_vec= np.linalg.inv(K+alpha*np.eye(N))@ y
    y_pred= K@ alpha_vec
    mse= np.mean((y_pred- y)**2)
    return mse

##############################################################################
# 9) The main training function + usage
##############################################################################

def train_classifier_and_get_kernels(
    n_qubits=4,
    n_layers=2,
    Nx_train=6, Ny_train=6,
    Nx_test=10, Ny_test=10,
    j1_range=(-4,4), j2_range=(-4,4),
    max_steps=30,
    batch_size=4,
    lr=0.02
):
    """
    1) build & train classifier with measure qubit0 => label=1 if above_l1 & below_l2
    2) build kernel_full, kernel_qubit for partial-wire
    returns (classifier_qnode, final_params, kernel_full, kernel_qubit0)
    """

    # A) create QNode, build data
    classifier_qnode, total_params= create_classifier_qnode(n_qubits, n_layers)
    states_train, labels_train, _, _= generate_data_grid(
        n_qubits, Nx_train, Ny_train,
        j1_range[0], j1_range[1], j2_range[0], j2_range[1],
        label_func=binary_label_function
    )
    states_test, labels_test, j1_test, j2_test= generate_data_grid(
        n_qubits, Nx_test, Ny_test,
        j1_range[0], j1_range[1], j2_range[0], j2_range[1],
        label_func=binary_label_function
    )

    rng= np.random.default_rng(1)
    params= 0.1*rng.normal(size=(total_params,))
    opt= qml.AdamOptimizer(lr)

    # mini-batch training
    N_train= len(states_train)
    all_indices= np.arange(N_train)
    num_batches= max(1, N_train//batch_size)

    def cost_batch(p, idx_batch):
        bsize= len(idx_batch)
        sum_p= 0.0
        for i in idx_batch:
            lbl= labels_train[i]
            p2= classifier_qnode(p, states_train[i])
            sum_p+= p2[int(lbl)]
        return 1.0 - (sum_p/bsize)

    for step in range(max_steps):
        rng.shuffle(all_indices)
        cost_sum= 0.0
        for b in range(num_batches):
            idx_b= all_indices[b*batch_size:(b+1)*batch_size]
            params, cval= opt.step_and_cost(lambda pp: cost_batch(pp, idx_b), params)
            cost_sum+= cval
        if step%5==0:
            ctrain= cost_fn(params, classifier_qnode, states_train, labels_train)
            acc_train= accuracy(params, classifier_qnode, states_train, labels_train)
            acc_test= accuracy(params, classifier_qnode, states_test, labels_test)
            print(f"Step {step} => cost={ctrain:.4f}, TrainAcc={acc_train:.3f}, TestAcc={acc_test:.3f}")

    final_cost= cost_fn(params, classifier_qnode, states_train, labels_train)
    final_train_acc= accuracy(params, classifier_qnode, states_train, labels_train)
    final_test_acc= accuracy(params, classifier_qnode, states_test, labels_test)
    print(f"Done => final cost={final_cost:.4f}, TrainAcc={final_train_acc:.3f}, TestAcc={final_test_acc:.3f}")

    # B) build kernel_full, kernel_qubit0
    kernel_full= build_kernel_full(n_qubits, n_layers, params)
    kernel_q0= build_kernel_qubit(n_qubits, n_layers, params, qubit_index=[0])
    kernel_q1= build_kernel_qubit(n_qubits, n_layers, params, qubit_index=[1])
    kernel_q2= build_kernel_qubit(n_qubits, n_layers, params, qubit_index=[2])
    kernel_q01= build_kernel_qubit(n_qubits, n_layers, params, qubit_index=[0,1])
    kernel_q12= build_kernel_qubit(n_qubits, n_layers, params, qubit_index=[1,2])
    kernel_q02= build_kernel_qubit(n_qubits, n_layers, params, qubit_index=[0,2])
    kernel_lst = [kernel_q0, kernel_q1, kernel_q2, kernel_q01, kernel_q12, kernel_q02]

    return classifier_qnode, params, kernel_full, kernel_lst


def f_of_x(j1, j2, n_qubits, classifier_qnode, params):
    """
    define y => circuit's probability of label=1
    ignoring actual label
    """
    st= ground_state_exact(n_qubits, j1, j2)
    p2= classifier_qnode(params, st)
    return p2[1]

def f_of_x01(J1, J2, n_qubits, classifier_qnode, params):
    """
    define y => circuit's probability of label=1
    ignoring actual label
    """
    below_l1= (J2 < -1 - J1)
    above_l1= (J2 > -1 - J1)
    above_l2= (J2 >  J1 - 1)
    below_l2= (J2 <  J1 - 1)
    # ignoring l3=1

    if (above_l1 and below_l2):
        return 1
    else:
        return 0

if __name__=="__main__":
    print("=== Minimal changes to use Nx2 for kernel inputs ===")
    n_qubits=3 #4
    n_layers=2

    # 1) train
    classifier_qnode, final_params, kernel_full, kernel_lst = train_classifier_and_get_kernels(
        n_qubits=n_qubits,
        n_layers=n_layers,
        Nx_train=6, Ny_train=6,
        Nx_test=10,  Ny_test=10,
        j1_range=(-4,4),
        j2_range=(-4,4),
        max_steps=0,
        batch_size=4,
        lr=0.02
    )
    kernel_q0, kernel_q1, kernel_q2, kernel_q01, kernel_q12, kernel_q02 = kernel_lst

    # 2) build random dataset => Nx2 => (J1,J2)
    rng2= np.random.default_rng(123)
    Ndata= 20
    X_rand= np.zeros((Ndata,2), dtype=float)
    y_rand= np.zeros(Ndata, dtype=float)
    for i in range(Ndata):
        j1= rng2.uniform(-4,4)
        j2= rng2.uniform(-4,4)
        val= f_of_x01(j1, j2, n_qubits, classifier_qnode, final_params) + rng2.normal(0,0.1)
        X_rand[i,0]= j1
        X_rand[i,1]= j2
        y_rand[i]= val

    # 3) apply kernel_full => kernel ridge => MSE
    # mse_full= kernel_ridge_mse(X_rand, y_rand, kernel_full, alpha=1e-6)
    # print(f"MSE(full-wire) = {mse_full:.5f}")

    # # 4) partial-wire qubit0 => MSE
    # mse_q0= kernel_ridge_mse(X_rand, y_rand, kernel_q0, alpha=1e-6)
    # print(f"MSE(part-wire[0]) = {mse_q0:.5f}")

    # # 4) partial-wire qubit0 => MSE
    # mse_q0= kernel_ridge_mse(X_rand, y_rand, kernel_q2, alpha=1e-6)
    # print(f"MSE(part-wire[2]) = {mse_q0:.5f}")

    # # 4) partial-wire qubit0 => MSE
    # mse_q0= kernel_ridge_mse(X_rand, y_rand, kernel_q01, alpha=1e-6)
    # print(f"MSE(part-wire[0,1]) = {mse_q0:.5f}")

    for k in kernel_lst:
      print(kernel_ridge_mse(X_rand, y_rand, k, alpha=1e-6))
    print(kernel_ridge_mse(X_rand, y_rand, kernel_full, alpha=1e-6))


# P greedy code

In [ ]:
import numpy as np
import math




##############################################################################
# 3. P-greedy ALGORITHM with fixed max_m dimension
##############################################################################
def p_greedy_newton_basis_maxM(kernel_func, Omega_b, max_m):
    """
    Runs P-greedy up to 'max_m' points, returning:
      - X_indices: list of chosen center indices
      - Nvals    : shape (max_m, nB) array
                   Nvals[m_idx, i] = N_{m_idx+1}(Omega_b[i]).
      - P2       : final array of P^2_{max_m}(x_i)
    """
    nB = len(Omega_b)
    if nB == 0:
        return [], np.zeros((max_m, 0)), []

    # Precompute diagonal: K(x_i, x_i)
    diagK = np.array([kernel_func(Omega_b[i], Omega_b[i]) for i in range(nB)])
    Nvals = np.zeros((max_m, nB))

    # 1) pick first center
    i1 = np.argmax(diagK)
    X_indices = [i1]

    denom_first = math.sqrt(diagK[i1]) if diagK[i1] > 1e-14 else 1e-14
    for i in range(nB):
        Nvals[0, i] = kernel_func(Omega_b[i], Omega_b[i1]) / denom_first

    # Initialize power-function squared
    P2 = diagK - Nvals[0,:]**2

    m = 1
    while m < max_m:
        # pick next center
        i_next = np.argmax(P2)
        X_indices.append(i_next)

        denom = math.sqrt(P2[i_next]) if P2[i_next] > 1e-14 else 1e-14

        # Build next Newton basis
        for i in range(nB):
            val = kernel_func(Omega_b[i], Omega_b[i_next])
            tmp = 0.0
            for k in range(m):
                tmp += Nvals[k, i_next]*Nvals[k, i]
            val -= tmp
            Nvals[m, i] = val / denom

        # update P^2
        P2 = P2 - Nvals[m,:]**2
        m += 1

    return X_indices, Nvals, P2


##############################################################################
# 4. Build an approximate-kernel function from the P-greedy basis
##############################################################################
def build_approx_kernel(Omega_b, Nvals, m):
    """
    Returns a function approx_kernel(x, y) that:
      approx_kernel(x_i, x_j) = sum_{k=0..m-1} Nvals[k, i]*Nvals[k, j]
    provided x_i, x_j are exactly in Omega_b.
    """
    nB = len(Omega_b)

    # Map each mesh point -> index
    # (We store them as tuples for dictionary keys.)
    index_map = {}
    for i in range(nB):
        # caution: floating comparisons. For many bandit tasks this is okay if
        # we always pass EXACT points from Omega_b. If new points appear, you'll need
        # extra logic to evaluate Nvals on them.
        index_map[tuple(Omega_b[i])] = i

    def compute_approx_kernel(x, y):
        """
        Return the approximate kernel for x,y in Omega_b
        """
        i = index_map[tuple(x)]
        j = index_map[tuple(y)]
        val = 0.0
        for k in range(m):
            val += Nvals[k, i]*Nvals[k, j]
        return val

    Kernel = np.zeros((nB, nB))
    for i in Omega_b:
        for j in Omega_b:
            Kernel[index_map[tuple(i)], index_map[tuple(j)]] = compute_approx_kernel(i, j)

    def approx_kernel(x,y):
      return Kernel[index_map[tuple(x)], index_map[tuple(y)]]


    return approx_kernel


##############################################################################
# 5. The function you requested: get_approximate_kernel_using_P_greedy(dim)
##############################################################################
def get_approximate_kernel_using_P_greedy(dim):
    """
    1) Construct a 3D mesh (10x10x10) as candidate set Omega_b.
    2) Run p_greedy_newton_basis_maxM(..., max_m=dim).
    3) Build and return approx_kernel(x,y) for x,y in Omega_b.
    """
    # This is your discrete domain for P-greedy:
    Omega_b = build_grid([(-4,4),(-4,4)],20)

    # Run P-greedy up to 'dim' basis functions using your "full_kernel_classic"
    X_indices, Nvals, _ = p_greedy_newton_basis_maxM(full_kernel_classic, Omega_b, max_m=dim)

    # Build the approximate-kernel function
    approx_kernel_func = build_approx_kernel(Omega_b, Nvals, dim)

    # Return the callable that you can use inside 'kernell(X, Y)'
    return approx_kernel_func


##############################################################################
# 6. Demo usage (Optional)
##############################################################################
if __name__ == "__main__":
    full_kernel_classic = lambda x,y: kernel_full(np.array([x]),np.array([y]))[0][0]
    # Example: get an approximate kernel with dim=5
    approxK = get_approximate_kernel_using_P_greedy(11)

    # Compare the approximate kernel vs. the true kernel on a random pair:
    Omega_test = build_grid([(-4,4),(-4,4)],20)
    idxA, idxB = 53, 56
    xA, xB = Omega_test[idxA], Omega_test[idxB]

    true_val = full_kernel_classic(xA, xB)
    approx_val = approxK(xA, xB)
    print(f"True K: {true_val:.6f}, Approx K: {approx_val:.6f}")


# whole domain

In [ ]:
def f_of_x01(J1, J2, n_qubits, classifier_qnode, params):
    """
    define y => circuit's probability of label=1
    ignoring actual label
    """
    below_l1= (J2 < -1 - J1)
    above_l1= (J2 > -1 - J1)
    above_l2= (J2 >  J1 - 1)
    below_l2= (J2 <  J1 - 1)
    # ignoring l3=1

    if (above_l1 and below_l2):
        return 1
    else:
        return 0



w_random = np.array([1 for _ in range(27)])
# setup
qubits = 3
seed = 1
bounds = [(-4,4),(-4,4)]

# get kernels and true reward functions

dim_tried = []
regret_for_dim = []
std_regret_for_dim = []

quantum_f = lambda X: np.array([f_of_x01(X[i][0], X[i][1], 3, classifier_qnode, final_params) for i in range(X.shape[0])])

for dim in trange(1,11,1, desc="Number of projected kernels Used"):

  approx_kernel = get_approximate_kernel_using_P_greedy(dim)

  def kernell(X,Y):
    K = np.zeros((X.shape[0],Y.shape[0]))
    for i in range(X.shape[0]):
      for j in range(Y.shape[0]):
        K[i,j] = approx_kernel(X[i],Y[j])
    return K


  gp = GaussianProcessRegressor(kernel=kernell, alpha=1e-3)
  avg_best, best_hist_all,std_regret = run_multiple_experiments_gp_ucb(
  f=quantum_f,
  gp=gp,
  bounds=bounds,
  n_runs=30,
  n_iter=100,
  init_points=1,
  n_grid=20,
  beta_func=None,  # use the default inside gp_ucb_nd
  random_state=42,
  verbose=False,
  error = 1/(dim)
)


  dim_tried.append(dim)
  regret_for_dim.append(avg_best[-1])
  std_regret_for_dim.append(std_regret[-1])
  print(avg_best)




plt.errorbar(dim_tried,regret_for_dim,std_regret_for_dim,ls="-",
             marker='d',
             color="#009E73",
             alpha=1.0,
             capsize=4)
plt.title("Regret for Different Kernel Used")
plt.xlabel("Number of Projected Kernel Used for Modeling")
plt.ylabel("Regret for T=100")
plt.grid(True)
plt.legend()
plt.show()

print(regret_for_dim)
print(std_regret_for_dim)
